In [1]:
using TensorFlow
tf_version()

v"1.10.0"

# Graphs and Sessions

TensorFlow uses a **dataflow graph** to represent your computation in terms of
the dependencies between individual operations. This leads to a low-level
programming model in which you first define the dataflow graph, then create a
TensorFlow **session** to run parts of the graph across a set of local and
remote devices.

This guide will be most useful if you intend to use the low-level programming
model directly. Higher-level APIs such as `estimator.Estimator` and Keras
hide the details of graphs and sessions from the end user, but this guide may
also be useful if you want to understand how these APIs are implemented.

## Why dataflow graphs?

![](https://www.tensorflow.org/images/tensors_flowing.gif)

[Dataflow](https://en.wikipedia.org/wiki/Dataflow_programming) is a common
programming model for parallel computing. In a dataflow graph, the nodes
represent units of computation, and the edges represent the data consumed or
produced by a computation. For example, in a TensorFlow graph, the matrix multiplication operation would correspond to a single node with two incoming edges (the
matrices to be multiplied) and one outgoing edge (the result of the
multiplication).

Dataflow has several advantages that TensorFlow leverages when executing your
programs:

* **Parallelism.** By using explicit edges to represent dependencies between
  operations, it is easy for the system to identify operations that can execute
  in parallel.

* **Distributed execution.** By using explicit edges to represent the values
  that flow between operations, it is possible for TensorFlow to partition your
  program across multiple devices (CPUs, GPUs, and TPUs) attached to different
  machines. TensorFlow inserts the necessary communication and coordination
  between devices.

* **Compilation.** TensorFlow's [XLA compiler](https://www.tensorflow.org/performance/xla/) can
  use the information in your dataflow graph to generate faster code, for
  example, by fusing together adjacent operations.

* **Portability.** The dataflow graph is a language-independent representation
  of the code in your model. You can build a dataflow graph in Julia, store it
  in a [SavedModel](http://malmaud.github.io/TensorFlow.jl/latest/saving.html), and restore it in a C++ program for
  low-latency inference.

## What is a `Graph`?

A `Graph` contains two relevant kinds of information:

* **Graph structure.** The nodes and edges of the graph, indicating how
  individual operations are composed together, but not prescribing how they
  should be used. The graph structure is like assembly code: inspecting it can
  convey some useful information, but it does not contain all of the useful
  context that source code conveys.

* **Graph collections.** TensorFlow provides a general mechanism for storing
  collections of metadata in a `Graph`. The `TensorFlow.add_to_collection` function
  enables you to associate a list of objects with a key, and `get_collection` enables you to
  look up all objects associated with a key. Many parts of the TensorFlow
  library use this facility: for example, when you create a `Variable`, it
  is added by default to collections representing "variables" and
  "trainable variables". When you later come to create a `train.Saver` or
  `train.Optimizer`, the variables in these collections are used as the
  default arguments.

## Building a `Graph`

Most TensorFlow programs start with a dataflow graph construction phase. In this
phase, you invoke TensorFlow API functions that construct new `Operation`
(node) and `Tensor` (edge) objects and add them to a `Graph`
instance. TensorFlow provides a **default graph** that is an implicit argument
to all API functions in the same context.  For example:

* Calling `constant(42.0)` creates a single `Operation` that produces the
  value `42.0`, adds it to the default graph, and returns a `Tensor` that
  represents the value of the constant.

* Calling `x * y` creates a single `Operation` that multiplies
  the values of `Tensor` objects `x` and `y`, adds it to the default graph,
  and returns a `Tensor` that represents the result of the multiplication.

* Executing `v = Variable(0)` adds to the graph a `Operation` that will
  store a writeable tensor value that persists between `run` calls.
  The `Variable` object wraps this operation, and can be used [like a
  tensor](#tensor-like_objects), which will read the current value of the
  stored value. The `Variable` object also has methods such as
  `assign` and `assign_add` that
  create `Operation` objects that, when executed, update the stored value.
  (See [Variables](./TensorFlow%20Guide%20-%20Variables.ipynb) for more information about variables.)

* Calling `train.minimize` on a `train.Optimizer` will add operations and tensors to the
  default graph that calculates gradients, and return a `Operation` that,
  when run, will apply those gradients to a set of variables.

Most programs rely solely on the default graph. However,
see [Dealing with multiple graphs](#Programming-with-multiple-graphs) for more
advanced use cases. High-level APIs such as the `estimator.Estimator` API
manage the default graph on your behalf, and--for example--may create different
graphs for training and evaluation.

**⋆Note:** Calling most functions in the TensorFlow API merely adds operations
and tensors to the default graph, but **does not** perform the actual
computation. Instead, you compose these functions until you have a `Tensor`
or `Operation` that represents the overall computation--such as performing
one step of gradient descent--and then pass that object to a `Session` to
perform the computation. See the section "Executing a graph in a `Session`"
for more details.

## Naming operations

A `Graph` object defines a **namespace** for the `Operation` objects it
contains. TensorFlow automatically chooses a unique name for each operation in
your graph, but giving operations descriptive names can make your program easier
to read and debug. The TensorFlow API provides two ways to override the name of
an operation:

* Each API function that creates a new `Operation` or returns a new
  `Tensor` accepts an optional `name` argument. For example,
  `constant(42.0, name="answer")` creates a new `Operation` named
  `"answer"` and returns a `Tensor` named `"answer:0"`. If the default graph
  already contains an operation named `"answer"`, then TensorFlow would append
  `"_1"`, `"_2"`, and so on to the name, in order to make it unique.

* **Name scopes**: not yet supported in TensorFlow.jl.

Note that `Tensor` objects are implicitly named after the `Operation`
that produces the tensor as output. A tensor name has the form `"<OP_NAME>:<i>"`
where:

* `"<OP_NAME>"` is the name of the operation that produces it.
* `"<i>"` is an integer representing the index of that tensor among the
  operation's outputs (follows Julia's 1-based indexing convention).

## Placing operations on different devices

If you want your TensorFlow program to use multiple different devices, the
`with_device` context manager function provides a convenient way to request that all operations
created in a particular context are placed on the same device (or type of
device).

A **device specification** has the following form:

```
/job:<JOB_NAME>/task:<TASK_INDEX>/device:<DEVICE_TYPE>:<DEVICE_INDEX>
```

where:

* `<JOB_NAME>` is an alpha-numeric string that does not start with a number.
* `<DEVICE_TYPE>` is a registered device type (such as `GPU` or `CPU`).
* `<TASK_INDEX>` is a positive integer representing the index of the task in the job named `<JOB_NAME>`.
* `<DEVICE_INDEX>` is a positive integer representing the index of the
  device, for example, to distinguish between different GPU devices used in the
  same process.

**⋆Note**: The indices used by TensorFlow.jl follows the 1-based Julia convention (e.g. "gpu:1" is the first GPU).

You do not need to specify every part of a device specification. For example,
if you are running in a single-machine configuration with a single GPU, you
might use `with_device` to pin some operations to the CPU and GPU:

```julia
# Operations created outside either context will run on the "best possible"
# device. For example, if you have a GPU and a CPU available, and the operation
# has a GPU implementation, TensorFlow will choose the GPU.
weights = random_normal(...)

with_device("/device:CPU:1") do
    # Operations created in this context will be pinned to the CPU.
    img = decode_jpeg(read_file("img.jpg"))
end

with_device("/device:GPU:1") do
    # Operations created in this context will be pinned to the GPU.
    result = weights * img
end
```

**⋆Note**: distributed deployment of TensorFlow.jl not yet supported.

## Tensor-like objects

Many TensorFlow operations take one or more `Tensor` objects as arguments.
For example, `*` takes two `Tensor` objects, and `add_n` takes
a list of `n` `Tensor` objects. For convenience, these functions will accept
a **tensor-like object** in place of a `Tensor`, and implicitly convert it
to a `Tensor` using the `convert` method. Tensor-like objects
include elements of the following types:

* `Tensor`
* `Variable`
* `Array`
* Lists of tensor-like objects
* Scalar Julia types: `Bool`, `Float64`, `Int64`, `String`

You can register additional tensor-like types by extending the `Base.convert(::Type{Tensor}, value)` method.

**⋆Note**: By default, TensorFlow will create a new `Tensor` each time you use
the same tensor-like object. If the tensor-like object is large (e.g. an
`Array` containing a set of training examples) and you use it multiple
times, you may run out of memory. To avoid this, convert the tensor-like object to `Tensor` once and use the returned `Tensor` instead. Conversion can be done either by `convert(Tensor, value)` or `Tensor(value)`.

## Executing a graph in a `Session`

TensorFlow.jl uses the `Session` class to represent a connection between the
client program---typically a Julia program, although a similar interface is
available in other languages---and the C++ runtime. A `Session` object
provides access to devices in the local machine, and remote devices using the
distributed TensorFlow runtime. It also caches information about your
`Graph` so that you can efficiently run the same computation multiple times.

### Creating a `Session`

If you are using the low-level TensorFlow API, you can create a `Session`
for the current default graph as follows:

```julia
# Create a default in-process session.
sess = Session()
# ...
close(sess)

# Create a remote session.
sess = Session("grpc://example.org:2222")
# ...
close(sess)
```

Since a `Session` owns physical resources (such as GPUs and network connections) and TensorFlow.jl does not yet have context manager support (Python `with` block) for sessions, you should remember to `close` the session when you are finished with it to free the resources.

**⋆Note:** Higher-level APIs such as `train.MonitoredTrainingSession` or
`estimator.Estimator` will create and manage a `Session` for you. These
APIs accept optional `target` and `config` arguments (either directly, or as
part of a `estimator.RunConfig` object), with the same meaning as
described below.

The `Session` constructor `Session(graph, config=nothing; target=nothin)` accepts three optional arguments:

* **`graph`.** By default, a new `Session` will be bound to---and only able
  to run operations in---the current default graph. If you are using multiple
  graphs in your program (see [Programming with multiple
  graphs](#Programming-with-multiple-graphs) for more details), you can specify
  an explicit `Graph` when you construct the session.

* **`config`.** This argument allows you to specify a `TensorFlow.tensorflow.ConfigProto` that
  controls the behavior of the session. For example, some of the configuration
  options include:

    * `allow_soft_placement`. Set this to `true` to enable a "soft" device
    placement algorithm, which ignores `device` annotations that attempt
    to place CPU-only operations on a GPU device, and places them on the CPU
    instead.

    * `cluster_def` (not yet supported by TensorFlow.jl)

    * `graph_options.optimizer_options`. Provides control over the optimizations
    that TensorFlow performs on your graph before executing it.

    * `gpu_options.allow_growth`. Set this to `True` to change the GPU memory
    allocator so that it gradually increases the amount of memory allocated,
    rather than allocating most of the memory at startup.

* **`target`.** If this argument is left empty (the default), the session will
  only use devices in the local machine. However, you may also specify a
  `grpc://` URL to specify the address of a TensorFlow server, which gives the
  session access to all devices on machines that this server controls. See
  `train.Server` for details of how to create a TensorFlow
  server. For example, in the common **between-graph replication**
  configuration, the `Session` connects to a `train.Server` in the same
  process as the client. The [distributed TensorFlow](https://www.tensorflow.org/deploy/distributed)
  deployment guide describes other common scenarios.

### Using `run` to execute operations

The `run` method is the main mechanism for running a `Operation`
or evaluating a `Tensor`. You can pass one or more `Operation` or
`Tensor` objects to `run`, and TensorFlow will execute the
operations that are needed to compute the result.

`run` requires you to specify a list of **fetches**, which determine
the return values, and may be a `Operation`, a `Tensor`, or
a [tensor-like type](#Tensor-like-objects) such as `Variable`. These fetches
determine what **subgraph** of the overall `Graph` must be executed to
produce the result: this is the subgraph that contains all operations named in
the fetch list, plus all operations whose outputs are used to compute the value
of the fetches. For example, the following code fragment shows how different
arguments to `run` cause different subgraphs to be executed:

In [2]:
x = constant([37.0 -23.0; 1.0 4.0])
w = random_uniform([2, 2], dtype=Float64)
y = x * w
output = nn.softmax(y)
sess = Session()

# Evaluate `output`. `run(sess, output)` will return an array containing the result of the computation.
println("output = ", run(sess, output))

# Evaluate `y` and `output`. Note that `y` will only be computed once, and its
# result used both to return `y_val` and as an input to the `nn.softmax()`
# op. Both `y_val` and `output_val` will be arrays.
y_val, output_val = run(sess, [y, output])
println("y_val = $y_val")
println("output_val = $output_val")

close(sess)

output = 

2018-10-04 23:22:06.781998: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


[1.18755e-8 1.0; 0.429209 0.570791]
y_val = [7.22399 -8.49654; 1.51007 2.83216]
output_val = [1.0 1.48819e-7; 0.21047 0.78953]


`run` also optionally takes a dictionary of **feeds**, which is a
mapping from `Tensor` objects (typically `placeholder` tensors) to
values (typically Julia scalars or arrays) that will be
substituted for those tensors in the execution. For example:

In [3]:
# Define a placeholder that expects a vector of three floating-point values,
# and a computation that depends on it.
x = placeholder(Float64, shape=[3])
y = x .^ 2
sess = Session()

# Feeding a value changes the result that is returned when you evaluate `y`.
println(run(sess, y, Dict(x => [1.0, 2.0, 3.0]))) # => "[1.0, 4.0, 9.0]"
println(run(sess, y, Dict(x => [0.0, 0.0, 5.0]))) # => "[0.0, 0.0, 25.0]" 

# Raises an error, because you must feed a value for a `placeholder()` when
# evaluating a tensor that depends on it.
# run(sess, y)

# Raises an error, because the shape of `37.0` does not match the shape
# of placeholder `x`.
# run(sess, y, Dict(x => 37.0))

close(sess)

[1.0, 4.0, 9.0]
[0.0, 0.0, 25.0]


**⋆Note**: optional arguments `options` and `run_metadata` for `run` not yet supported in TensorFlow.jl.

## Visualizing your graph

TensorFlow includes tools that can help you to understand the code in a graph.
The **graph visualizer** is a component of TensorBoard that renders the
structure of your graph visually in a browser. The easiest way to create a
visualization is to pass a `Graph` when creating the
`TensorFlow.summary.FileWriter`:

```julia
# Build your graph.
x = constant([37.0 -23.0; 1.0, 4.0])
w = random_uniform([2, 2])
y = x * w
# ...
loss = ...
optimizer = train.AdamOptimizer()
train_op = train.minimize(optimizer, loss)

sess = Session()
# `sess.graph` provides access to the graph used in a `Session`.
writer = TensorFlow.summary.FileWriter("/tmp/log/..."; graph=sess.graph)
# Perform your computation...
for i = 1:1000
    run(sess, train_op)
    # ...
close(writer)
```

You can then open the log in `tensorboard`, navigate to the "Graph" tab, and
see a high-level visualization of your graph's structure. Note that a typical
TensorFlow graph---especially training graphs with automatically computed
gradients---has too many nodes to visualize at once. The graph visualizer makes
use of name scopes to group related operations into "super" nodes. You can
click on the orange "+" button on any of these super nodes to expand the
subgraph inside.

![](https://www.tensorflow.org/images/mnist_deep.png)

For more information about visualizing your TensorFlow application with
TensorBoard, see the [TensorBoard guide](https://www.tensorflow.org/guide/summaries_and_tensorboard).

## Programming with multiple graphs

**⋆Note**: When training a model, a common way of organizing your code is to use one
graph for training your model, and a separate graph for evaluating or performing
inference with a trained model. In many cases, the inference graph will be
different from the training graph: for example, techniques like dropout and
batch normalization use different operations in each case. Furthermore, by
default utilities like `train.Saver` use the names of `Variable` objects
(which have names based on an underlying `Operation`) to identify each
variable in a saved checkpoint. When programming this way, you can either use
completely separate Julia processes to build and execute the graphs, or you can
use multiple graphs in the same process. This section describes how to use
multiple graphs in the same process.

As noted above, TensorFlow provides a "default graph" that is implicitly passed
to all API functions in the same context. For many applications, a single graph
is sufficient. However, TensorFlow also provides methods for manipulating
the default graph, which can be useful in more advanced use cases. For example:

* A `Graph` defines the namespace for `Operation` objects: each
  operation in a single graph must have a unique name. TensorFlow will
  "uniquify" the names of operations by appending `"_2"`, `"_3"`, and so on to
  their names if the requested name is already taken. Using multiple explicitly
  created graphs gives you more control over what name is given to each
  operation.
  
* The default graph stores information about every `Operation` and
  `Tensor` that was ever added to it. If your program creates a large number
  of unconnected subgraphs, it may be more efficient to use a different
  `Graph` to build each subgraph, so that unrelated state can be garbage
  collected.
  
You can install a different `Graph` as the default graph, using the
`as_default` context manager:

In [22]:
g_1 = Graph()
as_default(g_1) do
    # Operations created in this scope will be added to `g_1`.
    global c = constant("Node in g_1")
    
    # Sessions created in this scope will run operations from `g_1`.
    global sess_1 = Session()
end

g_2 = Graph()
as_default(g_2) do
    # Operations created in this scope will be added to `g_2`.
    global d = constant("Node in g_2")
end

# Alternatively, you can pass a graph when constructing a `Session`:
# `sess_2` will run operations from `g_2`.
sess_2 = Session(graph=g_2)
@assert get_graph(c) === g_1
@assert sess_1.graph === g_1
@assert get_graph(d) === g_2
@assert sess_2.graph === g_2

To inspect the current default graph, call `get_default_graph`, which
returns a `Graph` object:

In [31]:
# Print all of the operations in the default graph.
g = get_def_graph()
println(get_operations(g))

TensorFlow.OperationIterator(Graph(Ptr{Nothing} @0x0000000005031550))


----------------------
*Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 3.0 License](https://creativecommons.org/licenses/by/3.0/), and code samples are licensed under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). This notebook is adapted from the official [TensorFlow Guide on Graphs and Sessions](https://www.tensorflow.org/guide/graphs).*